# Multiple Linear Regression Model #

### Time Series Regression Explaining Stock's Returns ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visuals
import matplotlib.pyplot as plt

# Statistics
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import t

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from config import get_tickers
from data_downloader import get_market_data

The Multiple Linear Regression Model implies matrix notation:

1) $ \mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \boldsymbol{\varepsilon} $

In [2]:
tickers = get_tickers(mod="3.2")

tickers

In [5]:
# Import data
data_regression = pd.DataFrame()

for ticker in tickers[:-1]:
    df = get_market_data(
        ticker=ticker, 
        start_date='2015-01-01', 
        end_date='2025-01-01', 
        returns=True
    )
    
    returns = df['returns'].rename(ticker)
    
    data_regression = pd.concat([data_regression, returns], axis=1)
    
    print(f'Data Ready for {ticker}')

In [7]:
data_regression.columns = ['stock_returns', 'mkt_returns', 'sector_returns', 'dollar_returns']

data_regression

In [59]:
# 10-Year Bond Yield
data_tnx = get_market_data(
    ticker=tickers[-1], 
    start_date='2015-01-01', 
    end_date='2025-01-01', 
    returns=False
)

data_tnx

In [9]:
# Function to calculate daily rate
def annual_to_daily_rate(
        series, 
        days_per_year=252
):
    # From % to decimal
    decimal_series = series / 100
    
    #Calculate the Rate
    return (1 + decimal_series) ** (1 / days_per_year) - 1

In [10]:
# Calculate daily rate
data_tnx['daily_rate'] = annual_to_daily_rate(data_tnx['close'])

data_tnx

In [11]:
# Add the Daily Rate
data_regression['10_year_bond'] = data_tnx['daily_rate']

# Drop nans
data_regression.dropna(inplace = True)

data_regression

In [12]:
# Make a Plot
fig, axs = plt.subplots(2, 2, figsize=(12, 8), dpi=600, sharex=True)  # 2 rows, 2 columns

# First Graph
data_regression['mkt_returns'].cumsum().plot(ax=axs[0, 0], color='blue', label='Market Returns')
axs[0, 0].set_title('Returns')
axs[0, 0].legend()

# Second Graph
data_regression['sector_returns'].cumsum().plot(ax=axs[0, 1], color='orange', label='Sector Returns')
axs[0, 1].set_title('Returns')
axs[0, 1].legend()

# Third Graph
data_regression['dollar_returns'].cumsum().plot(ax=axs[1, 0], color='green', label='Dollar Returns')
axs[1, 0].set_title('Returns')
axs[1, 0].legend()

# Fourth
data_regression['10_year_bond'].plot(ax=axs[1, 1], color='red', label='10-Year Bond')
axs[1, 1].set_title('Rate')
axs[1, 1].legend()

# Show
plt.tight_layout()
plt.show()

In [13]:
# Correlation Matrix
data_regression.corr()

In [15]:
# Setup subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 8), dpi=600)

# Variables to plot against 'returns'
variables = ['mkt_returns', 'sector_returns', 'dollar_returns', '10_year_bond']
titles = ['Market Returns', 'Sector Returns', 'Dollar Returns', '10-Year Bond']
colors = ['orange', 'green', 'red', 'purple']

# Create scatter plots
for i, (var, title, color) in enumerate(zip(variables, titles, colors)):
    row, col = divmod(i, 2)
    axs[row, col].scatter(data_regression[var], data_regression['stock_returns'], alpha=0.5, color=color)
    axs[row, col].set_title(f'Returns vs {title}')
    axs[row, col].set_xlabel(title)
    axs[row, col].set_ylabel('Returns')

# Layout and show
plt.tight_layout()
plt.show()

In [16]:
filtered_data = data_regression[(np.abs(data_regression - data_regression.mean()) <= 3 * data_regression.std())]

filtered_data.dropna(inplace = True)

filtered_data

In [17]:
# The Correlation Matrix
filtered_data.corr()

In [19]:
# Create the Y Vector
Y_Vector = filtered_data["stock_returns"]

Y_Vector

In [20]:
# Create the Information Matrix
Information_Matrix = filtered_data.copy().drop('stock_returns', axis=1)

# Add a constant
Information_Matrix = sm.add_constant(Information_Matrix)

Information_Matrix

In [21]:
# For estimating the coefficients with the OLS we need to assess the non-perfect collinearity condition with the matrix rank
Rank = np.linalg.matrix_rank(Information_Matrix)

# Since we are estimating four betas, the rank must be the number of columns in the Information Matrix
if Rank == len(Information_Matrix.columns):
    print(f"Matrix Rank is {Rank}; there is no evidence of Perfect Multicollinearity.")
else:
    print(f"Matrix Rank is {Rank}; there is evidence of Perfect Multicollinearity between two or more variables.")

The beta calculation in the Matrix Form:
1) $ \boldsymbol{\beta}=(\mathbf{X}^⊤\mathbf{X})^{-1}(\mathbf{X}^⊤\mathbf{y}) $

The vector of betas is a kx1 vector containing the coefficients of the regression model, where k is the number of parameters to estimate

In [22]:
# Transpose of the Information Matrix
Information_Matrix_T = Information_Matrix.transpose()

Information_Matrix_T

In [23]:
# Now we have to calculate the (X^⊤X)^{-1}, first the square of the Information Matrix
Information_Matrix_Square = Information_Matrix_T.dot(Information_Matrix)

# We could also use the command Information_Matrix_T @ Information_Matrix
Information_Matrix_Square

In [24]:
# The reason we needed to know the rank of the information matrix is because, only full rank matrix can be invertible
# Then we can calculate the Matrix Determinant, if it is different from zero, we can calculate the OLS coefficients
Information_Matrix_Square_Determinant = np.linalg.det(Information_Matrix_Square)

if Information_Matrix_Square_Determinant != 0:
    print(f"The Determinant of the Squared Information Matrix is {Information_Matrix_Square_Determinant} and different from zero")
else:
    print("Matrix NOT invertible")

In [25]:
# Now we have to get the Inverse Matrix
X_Variance_Matrix_Inverse = np.linalg.inv(Information_Matrix_Square)

X_Variance_Matrix_Inverse

In [26]:
# Now we have to obtain (X^⊤Y)
Y_Covariance_X = Information_Matrix_T.dot(Y_Vector)

Y_Covariance_X

In [27]:
# Now we can calculate the Betas
Beta = X_Variance_Matrix_Inverse.dot(Y_Covariance_X)

Beta_DF = pd.DataFrame(Beta, index = Information_Matrix.columns)
Beta_DF

In [28]:
# Now we can get the fitted values
Y_Hat = Information_Matrix.dot(Beta)

Y_Hat

In [30]:
fig, ax1 = plt.subplots(dpi = 600)

filtered_data['expected_returns'] = Y_Hat

filtered_data['stock_returns'].cumsum().plot(label = 'Observed')
filtered_data['expected_returns'].cumsum().plot(label = 'Fitted')
plt.legend()
plt.title('Observed vs Fitted')

plt.show()

In [31]:
# We can get the Hat Matrix, which is a matrix used to transform the real values into the fitted values
Some_Matrix = Information_Matrix.dot(X_Variance_Matrix_Inverse)
Hat_Matrix = Some_Matrix.to_numpy() @ Information_Matrix_T.to_numpy()

Hat_Matrix

In [32]:
print(Information_Matrix.shape)
print(X_Variance_Matrix_Inverse.shape)
print(Some_Matrix.shape)
print(Information_Matrix_T.shape)

In [33]:
# Let us check if this is true
Y_Hat_2 = Hat_Matrix.dot(Y_Vector)

Y_Hat_2

In [34]:
# Hat Matrix is Symmetric and Idempotent
Hat_Matrix_Square = (Hat_Matrix.transpose()).dot(Hat_Matrix)

if Hat_Matrix.all() == Hat_Matrix_Square.all():
    print("It is indeed idempotent")
else:
    print("Wrong!")

In [35]:
# We can calculate the residuals using the Hat Matrix
Identity_Matrix = np.identity(len(Y_Vector))
Residuals_Vector = (Identity_Matrix - Hat_Matrix).dot(Y_Vector)

print(f"The Residuals Mean is: {Residuals_Vector.mean().round(3)}")
print(f"The Residuals Variance is: {Residuals_Vector.var()}")

In [36]:
# Residual Returns
filtered_data['residuals'] = Residuals_Vector

filtered_data

In [37]:
# Plot
fig, ax1 = plt.subplots(dpi = 600)
filtered_data['residuals'].cumsum().plot()
plt.title('Residual Returns Time Series')
plt.show()

In [38]:
# The OLS Assumptions establish that the covariances and the residuals must be uncorrelated
Intercorrelation_Vector = Information_Matrix_T.dot(Residuals_Vector)

print(Intercorrelation_Vector.round(5))

We can use this expression to calculate the Bias of the Beta coefficients:

1) $ S=(\mathbf{X}^⊤\mathbf{X})^{-1}(\mathbf{X}^⊤\boldsymbol{\varepsilon}) $


In [39]:
# Calculate Bias
Bias = X_Variance_Matrix_Inverse.dot(Intercorrelation_Vector)

print("""
Biases are very close to zero.
""")
print(Bias)

Now we want to calculate the sum of squares

1) $ RSS=\boldsymbol{\varepsilon}^⊤\boldsymbol{\varepsilon}=\mathbf{y}^⊤\mathbf{y}-\boldsymbol{\beta}^⊤\mathbf{X}^⊤\mathbf{y} $
2) $ ESS=\boldsymbol{\beta}^⊤\mathbf{X}^⊤\mathbf{y}-\bar{\mathbf{y}}^2 $
3) $ TSS=\mathbf{y}^⊤\mathbf{y}-\bar{\mathbf{y}}^2 $

In [40]:
RSS = (Residuals_Vector.transpose()).dot(Residuals_Vector)
ESS = (Beta.transpose()).dot(Y_Covariance_X) - (sum(Y_Vector)**2)/len(Y_Vector)
TSS = (Y_Vector.transpose()).dot(Y_Vector) - (sum(Y_Vector)**2)/len(Y_Vector)

print(f"The Residuals Sum of Squares is: {RSS}")
print(f"The Estimation Sum of Squares is: {ESS}")
print(f"The Total Sum of Squares is: {TSS}")

In [41]:
# We can calculate the R-Squared Coefficient
R_Squared = ESS/TSS

print(f"The R-Squared Coefficient is: {R_Squared}")

In [42]:
# Now calculate the Residual Variance with n - k degrees of freedom (adjusted to the sample)
Residuals_Variance = RSS/(len(Y_Vector) - Hat_Matrix.trace())

print(f"The Residuals Variance is: {Residuals_Variance}")

We use the Residual Variance to calculate the covariances between all the beta coefficients:

1) $ C=\frac{\boldsymbol{\varepsilon}^⊤\boldsymbol{\varepsilon}}{n-k}(\mathbf{X}^⊤\mathbf{X})^{-1} $

In [43]:
# The Diagonal of the Covariance Matrix contains the standard errors of the beta coefficients
Covariance_Matrix = Residuals_Variance * X_Variance_Matrix_Inverse

Covariance_Matrix

In [44]:
# Take the squared-root
Beta_Standards_Errors = np.sqrt(Covariance_Matrix.diagonal())

Beta_Standards_Errors

In [45]:
# Calculate the T-Values
T_Values = Beta/Beta_Standards_Errors

T_Values

In [46]:
# How many degrees of freedom we have?
df = len(Y_Vector) - Hat_Matrix.trace()

print(f"We have {df.round()} degrees of freedom.")

In [47]:
# The Hypothesis Testing implies to reject the null hypothesis if the t-values are higher than the critic t-value
# For 293 degrees of freedom the critic t-value approaches to 1.96
# Then we can calculate the upper and lower limits

Beta_Lower_Limit = Beta - 1.96*Beta_Standards_Errors
Beta_Upper_Limit = Beta + 1.96*Beta_Standards_Errors

print(Beta_Lower_Limit)
print(Beta_Upper_Limit)

In [48]:
# We can build a dataframe that contains all the information
Proof_DF = pd.DataFrame(
    {
     "T_Values": T_Values, 
     "Beta_Inferior_Limit": Beta_Lower_Limit, 
     "Beta_Superior_Limit": Beta_Upper_Limit
     }
    )

Proof_DF

In [49]:
# Let us get the p-values, if these are less than 0.05, we reject the null hypothesis confirming statistical significance
Proof_DF["p-values"] = 2*(t.sf(
    abs(Proof_DF.T_Values), 
    len(Y_Vector) - Hat_Matrix.trace()
    ).round(3)
    )

Proof_DF

In [50]:
"""
The R-Squared is not always the most precise statistic in a multivariate model
The Adjusted R-Squared penalizes the existence of more variables in our model:
    
    Adjusted R Squared = 1 – [((1 – R2) * (n – 1)) / (n – k)]
    
"""
R_Squared_Adjusted = (1 - ((1-R_Squared)*(len(Y_Vector) - 1)/(len(Y_Vector) - Hat_Matrix.trace())))

print(f"The Adjusted R-Squared is: {R_Squared_Adjusted.round(5)}")

The F-Statistic helps us to prove Joint Significance.
This means that we are checking if our models as a whole can explain the Y

The F statistic is distributed in an F-distribution with n - k and
k - 1 degrees of freedom.


1) $ F=\frac{ESS/k-1}{RSS/n-k} $


In [51]:
# Calculate the F Stat
F_Stat = (ESS/(Hat_Matrix.trace() - 1)) / (RSS/(len(Y_Vector) - Hat_Matrix.trace()))

print(f"The F-Statistic is: {F_Stat}")

In [52]:
#Model specification
model = sm.OLS(
    Y_Vector, 
    sm.add_constant(Information_Matrix)
    )   
     
#the results of the model
results = model.fit() 
    
#The Parameters
Beta2 = results.params  

#here we check the summary
print(results.summary())       

In [53]:
Beta

In [54]:
# Calculate the VIF
X = Information_Matrix

vif_data = pd.DataFrame()
vif_data['vars'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data

In [55]:
r_squared_df = pd.DataFrame()
r_squared_df['vars'] = X.columns

r_squared_df['r_squared'] = 1 - (1 / vif_data['VIF'])

r_squared_df

In [56]:
#Model specification
model = sm.OLS(
    Y_Vector, 
    sm.add_constant(X.drop(['mkt_returns'], axis=1))
    )   
     
#the results of the model
results = model.fit() 

#here we check the summary
print(results.summary())   

In [57]:
# Calculate the VIF

X = Information_Matrix.drop(['mkt_returns'], axis=1)

vif_data = pd.DataFrame()
vif_data['vars'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data